### 使用autograd来自动求导

In [1]:
#先导入autograd
import mxnet.ndarray as nd
import mxnet.autograd as ag

### 为变量附上梯度

In [3]:
#假设我们想对函数 f=2×x2f=2×x2 求关于 xx 的导数。我们先创建变量x，并赋初值。
x = nd.array([[1, 2], [3, 4]])

In [4]:
#当进行求导的时候，我们需要一个地方来存x的导数，
#这个可以通过NDArray的方法attach_grad()来要求系统申请对应的空间。
x.attach_grad()

In [5]:
#下面定义f。默认条件下，MXNet不会自动记录和构建用于求导的计算图，
#我们需要使用autograd里的record()函数来显式的要求MXNet记录我们需要求导的程序
with ag.record():
    y = x * 2
    z = y * x

In [6]:
#接下来我们可以通过z.backward()来进行求导。
#如果z不是一个标量，那么z.backward()等价于nd.sum(z).backward().
z.backward()

In [7]:
#现在我们来看求出来的导数是不是正确的
print('x.grad: ', x.grad)
x.grad == 4*x

x.grad:  
[[  4.   8.]
 [ 12.  16.]]
<NDArray 2x2 @cpu(0)>



[[ 1.  1.]
 [ 1.  1.]]
<NDArray 2x2 @cpu(0)>

### 对控制流求导
命令式的编程的一个便利之处是几乎可以对任意的可导程序进行求导，即使里面包含了Python的控制流。考虑下面程序，里面包含控制流for和if，但循环迭代的次数和判断语句的执行都是取决于输入的值。不同的输入会导致这个程序的执行不一样。（对于计算图框架来说，这个对应于动态图，就是图的结构会根据输入数据不同而改变）

In [8]:
def f(a):
    b = a * 2
    while nd.norm(b).asscalar() < 1000:
        b = b * 2
    if nd.sum(b).asscalar() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [9]:
#我们可以跟之前一样使用record记录和backward求导。
a = nd.random_normal(shape=3)
a.attach_grad()
with ag.record():
    c = f(a)
c.backward()

In [10]:
a.grad == c/a


[ 1.  1.  1.]
<NDArray 3 @cpu(0)>